In [1]:
import pandas as pd
import numpy as np
import pickle
from surprise import SVD, KNNBaseline
from surprise import Reader
from surprise import Dataset

In [2]:
#加载数据
with open("data.pkl", "rb") as file:
    data = pickle.load(file)
# print(data.head())
#将数据集由dataframe转化为surprise数据格式
reader = Reader(rating_scale=(1,5))
dataset = Dataset.load_from_df(data, reader)
trainset = dataset.build_full_trainset()

#### user_CF

In [3]:
#找到最接近的personId
#训练模型
algo = KNNBaseline()
algo.fit(trainset)
#取id 30的personId
sample_id = 30
top_k = 10
person_raw_id = data["personId"].values[sample_id]
# print(personId)
#得到内部id
person_inner_id = algo.trainset.to_inner_uid(person_raw_id)
#得到最接近的10个邻居
person_neighbors = algo.get_neighbors(person_inner_id, k=top_k)
# print(personId_neighbors)
#把内部id转为原personId
person_neighbors = list(algo.trainset.to_raw_uid(inner_id) for inner_id in person_neighbors)
print("和{}最接近的{}个人的id为：\n{}".format(person_raw_id,top_k,person_neighbors))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
和-9223121837663643404最接近的10个人的id为：
[-9199575329909162940, -9172914609055320039, -9120685872592674274, -9109785559521267180, -8781635134606732409, -8781306637602263252, -8738496712327699923, -8672331451814079632, -8652741825481604192, -8550167523008133722]


In [4]:
#内部编码10号用户
person_inner_id = 10
person_ratings = trainset.ur[person_inner_id]
# print(person_ratings)
items = map(lambda x:x[0],person_ratings)
#内容内部id,预测评分
for item,rating in person_ratings:
    print(algo.predict(algo.trainset.to_raw_uid(person_inner_id), algo.trainset.to_raw_iid(item), r_ui=rating))    

user: -9063420486253202900 item: -5920475612630001479 r_ui = 1.81   est = 1.59   {'actual_k': 40, 'was_impossible': False}
user: -9063420486253202900 item: -1876558460087500339 r_ui = 1.81   est = 1.35   {'actual_k': 19, 'was_impossible': False}
user: -9063420486253202900 item: -1419188045265393093 r_ui = 2.00   est = 1.53   {'actual_k': 14, 'was_impossible': False}
user: -9063420486253202900 item: -912681925384819247 r_ui = 1.81   est = 1.49   {'actual_k': 15, 'was_impossible': False}
user: -9063420486253202900 item: 2277184202014276839 r_ui = 1.00   est = 1.33   {'actual_k': 34, 'was_impossible': False}
user: -9063420486253202900 item: 5036201777135800491 r_ui = 1.81   est = 1.36   {'actual_k': 13, 'was_impossible': False}


In [5]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """
    return the top_N recommendation for each user 
    
    """
    #取出每个用户的预测
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid,est))
    #排序得到topn
    for uid,user_ratings in top_n.items():
        user_ratings.sort(key=lambda x:x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [6]:
#生成所有训练集中没有的user_item对
testset = trainset.build_anti_testset()
with open("testset.pkl", "wb") as file:
    pickle.dump(testset, file)

#### 预测并按得分排序

In [7]:
predictions = algo.test(testset)
top_n = get_top_n(predictions, n=10)

In [8]:
#输出每个用户的预测列表
# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid,_) in user_ratings])

#### 使用矩阵分解进行预测

In [9]:
from surprise import NMF
#建模训练
algo = NMF()
algo.fit(trainset)

In [10]:
#取内部id 10的person查看预测结果
person_inner_id = 10
user_rating = trainset.ur[person_inner_id]
for item, rating in user_rating:
    print(algo.predict(algo.trainset.to_raw_uid(person_inner_id), algo.trainset.to_raw_iid(item), rating))

user: -9063420486253202900 item: -5920475612630001479 r_ui = 1.81   est = 1.69   {'was_impossible': False}
user: -9063420486253202900 item: -1876558460087500339 r_ui = 1.81   est = 1.74   {'was_impossible': False}
user: -9063420486253202900 item: -1419188045265393093 r_ui = 2.00   est = 1.88   {'was_impossible': False}
user: -9063420486253202900 item: -912681925384819247 r_ui = 1.81   est = 1.72   {'was_impossible': False}
user: -9063420486253202900 item: 2277184202014276839 r_ui = 1.00   est = 1.22   {'was_impossible': False}
user: -9063420486253202900 item: 5036201777135800491 r_ui = 1.81   est = 1.53   {'was_impossible': False}


In [11]:
#保存模型
import surprise
surprise.dump.dump("nmf.model", algo=algo)

#### 不同推荐系统算法模型评估

In [12]:
from surprise.model_selection import KFold
from surprise import accuracy

kf = KFold(n_splits =5)

In [13]:
from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore, SVD, SVDpp, SlopeOne

algo_names = ["NormalPredictor", "BaselineOnly",
             "KNNBasic","KNNWithMeans", "KNNWithZScore", 
             "SVD", "SVDpp", "SlopeOne","NMF", "KNNBaseline"]
algos = [NormalPredictor(), BaselineOnly(), 
         KNNBasic(), KNNWithMeans(), KNNWithZScore(), 
         SVD(), SVDpp(), SlopeOne(),NMF(), KNNBaseline()]

for algo_name,algo in zip(algo_names, algos):
    rmse = []
    for trainset, testset in kf.split(dataset):
        algo.fit(trainset)
        pred = algo.test(testset)
        rmse.append(accuracy.rmse(pred, verbose=False))
    print("{}模型的rmse:{}".format(algo_name, np.mean(rmse)))

NormalPredictor模型的rmse:0.8838844083408214
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
BaselineOnly模型的rmse:0.6206038171160144
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic模型的rmse:0.6796090525037926
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.